# Школа алготрейдеров. Блок торгового ПО и программирования
## Занятие 4. Использование библиотеки pandas. Получение финансовых данных из различных источников
## Notebook C. Полная версия

### Задача: загрузка данных с помощью [`pandas_datareader`](http://pandas.pydata.org/pandas-docs/stable/remote_data.html)

Необходимо загрузить данные торгов для акций Apple Inc. на NASDAQ за период с июня 2012 по август 2016 года. В таблице должны остаться колонки **Open**, **High**, **Low**, **Close** и **Volume**.

Установить пакет можно с помощью команды `conda install pandas-datareader`

**Подсказки**
1. Для загрузки данных достаточно импортировать модуль `data` из `pandas_datareader` (`from pandas_datareader import data`), а затем вызвать функцию `data.DataReader` с параметрами `'aapl'`, `'yahoo'` и датами начала и конца диапазона.
1. Чтобы оставить только нужные колонки, удалите **Adj Close** командой `del` (см. второе занятие).

In [9]:
from pandas_datareader import data

In [10]:
from datetime import datetime
yahoo_aapl = data.DataReader('aapl', 'yahoo', datetime(2012, 6, 1), datetime(2016, 8, 30))
del yahoo_aapl['Adj Close']
yahoo_aapl.head()

,Open,High,Low,Close,Volume
Date,,,,,
2012-06-01,569.159996,572.650009,560.520012,560.989983,130246900
2012-06-04,561.500008,567.499985,548.499977,564.289978,139248900
2012-06-05,561.269989,566.470001,558.330002,562.830025,97053600
2012-06-06,567.770004,573.849983,565.499992,571.460022,100363900
2012-06-07,577.290009,577.320023,570.500000,571.720001,94941700


### Задача: загрузка данных с помощью Quandl

Используя HTTP API сервиса [Quandl](https://www.quandl.com), загрузите данные торгов для акций Facebook Inc. из базы «Wiki EOD Stock Prices» за тот же период и с теми же колонками, что и выше.
Колонка с датой должна быть ключевой (index) и содержать именно даты, а не строки.
Даты должны быть отсортированы в прямом хронологическом порядке.

**Подсказки**
1. Функция `read_csv` модуля `pandas` может принимать URL (веб-адрес) вместо названия файла.
1. [Страница индекса](https://www.quandl.com/data/WIKI/FB-Facebook-Inc-FB-Prices-Dividends-Splits-and-Trading-Volume), код — `WIKI/FB`.
1. [Описание HTTP API](https://www.quandl.com/docs/api?csv#customize-your-dataset).
1. Используйте параметры `usecols`, `index_col` и `parse_dates` функции [`read_csv`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html).

In [11]:
import pandas as pd
quandl_fb = pd.read_csv(
    'https://www.quandl.com/api/v3/datasets/WIKI/FB.csv?start_date=2012-06-01&end_date=2016-08-30&order=asc',
    usecols=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'],
    index_col='Date',
    parse_dates=['Date']
)
quandl_fb.head()

,Open,High,Low,Close,Volume
Date,,,,,
2012-06-01,28.892,29.15,27.39,27.720,41855500.0
2012-06-04,27.200,27.65,26.44,26.900,35230300.0
2012-06-05,26.700,27.76,25.75,25.869,42473400.0
2012-06-06,26.070,27.17,25.52,26.810,61489200.0
2012-06-07,27.000,27.35,26.15,26.310,26159500.0


In [12]:
quandl_fb.index

DatetimeIndex(['2012-06-01', '2012-06-04', '2012-06-05', '2012-06-06',
               '2012-06-07', '2012-06-08', '2012-06-11', '2012-06-12',
               '2012-06-13', '2012-06-14',
               ...
               '2016-08-17', '2016-08-18', '2016-08-19', '2016-08-22',
               '2016-08-23', '2016-08-24', '2016-08-25', '2016-08-26',
               '2016-08-29', '2016-08-30'],
              dtype='datetime64[ns]', name='Date', length=1069, freq=None)

### Задача: объединение таблиц

Нужно объединить полученные ранее данные, используя общий ключ — дату. Колонки должны получить суффиксы `AAPL` и `FB`.

**Подсказка**

Используйте функцию [`join`](http://pandas.pydata.org/pandas-docs/stable/merging.html).

In [13]:
res = yahoo_aapl.join(quandl_fb, lsuffix='AAPL', rsuffix='FB')
res.head()

,OpenAAPL,HighAAPL,LowAAPL,CloseAAPL,VolumeAAPL,OpenFB,HighFB,LowFB,CloseFB,VolumeFB
Date,,,,,,,,,,
2012-06-01,569.159996,572.650009,560.520012,560.989983,130246900,28.892,29.15,27.39,27.720,41855500.0
2012-06-04,561.500008,567.499985,548.499977,564.289978,139248900,27.200,27.65,26.44,26.900,35230300.0
2012-06-05,561.269989,566.470001,558.330002,562.830025,97053600,26.700,27.76,25.75,25.869,42473400.0
2012-06-06,567.770004,573.849983,565.499992,571.460022,100363900,26.070,27.17,25.52,26.810,61489200.0
2012-06-07,577.290009,577.320023,570.500000,571.720001,94941700,27.000,27.35,26.15,26.310,26159500.0


### Задача: агрегация данных

Сагрегируйте данные по неделям наиболее разумным способом.

**Подсказка**

См. методы [`resample`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html), [`agg`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.agg.html) и предыдущее занятие.

In [14]:
aggregated = res.resample('1w').agg({
    'OpenAAPL': 'first',
    'OpenFB': 'first',
    'HighAAPL': 'max',
    'HighFB': 'max',
    'LowAAPL': 'min',
    'LowFB': 'min',
    'CloseAAPL': 'last',
    'CloseFB': 'last',
    'VolumeAAPL': 'sum',
    'VolumeFB': 'sum'
})
aggregated.head()

,HighAAPL,VolumeAAPL,OpenFB,LowAAPL,OpenAAPL,CloseFB,LowFB,VolumeFB,CloseAAPL,HighFB
Date,,,,,,,,,,
2012-06-03,572.650009,130246900,28.892,560.520012,569.159996,27.720,27.390,41855500.0,560.989983,29.15
2012-06-10,580.580017,518487200,27.200,548.499977,561.500008,27.100,25.520,203386400.0,580.319984,27.76
2012-06-17,588.499985,500263400,27.180,566.700020,587.719994,30.014,26.835,121558000.0,574.129990,30.10
2012-06-24,589.999992,442895600,29.955,570.370010,570.959984,33.050,29.410,186090800.0,582.099998,33.45
2012-07-01,584.000015,372063300,32.860,565.610008,577.299995,31.095,30.760,115019800.0,584.000015,33.44


### Задача: вычисление квантилей

Вычислите 10-ю, 50-ю и 90-ю перцентили колонок **VolumeAAPL** и **VolumeFB**.

**Подсказка**

Используйте метод [`quantile`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.quantile.html).

In [15]:
aggregated[['VolumeAAPL', 'VolumeFB']].quantile([.1, .5, .9])

,VolumeAAPL,VolumeFB
0.1,162087440.0,86535332.2
0.5,313459300.0,173091300.0
0.9,646188480.0,370151400.0


### Задача: вычисление скользящего среднего

**Подсказка**

Используйте метод [`rolling`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html).

In [16]:
roll = aggregated.rolling(window=10).mean()
roll = roll[roll.LowAAPL.notnull()]
roll.head()

,HighAAPL,VolumeAAPL,OpenFB,LowAAPL,OpenAAPL,CloseFB,LowFB,VolumeFB,CloseAAPL,HighFB
Date,,,,,,,,,,
2012-08-05,599.295004,445676420.0,29.2052,574.895006,584.711001,28.4984,27.0265,136972610.0,589.754999,30.271
2012-08-12,604.530002,463997660.0,28.4550,580.369005,589.524003,27.9071,26.3095,146337520.0,595.826000,29.601
2012-08-19,611.291000,451489920.0,27.9500,587.844007,595.713004,27.1021,25.6575,165899050.0,602.605002,29.070
2012-08-26,619.929003,472685990.0,27.1370,595.985005,601.942006,26.0416,24.8490,182158780.0,611.514001,28.073
2012-09-02,629.016004,466852750.0,26.0905,604.673006,612.845006,24.5424,23.7110,178715440.0,619.828002,26.681
